In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Garbage collector
import gc
gc.enable()

In [2]:
# Import the preprocessed dataframes
train_raw_df = pd.read_csv("(4)labeled_train_plus.csv",
    dtype={'fullVisitorId': str}, nrows=None)
test_raw_df = pd.read_csv("(4)labeled_test_plus.csv",
    dtype={'fullVisitorId': str}, nrows=None)
train_raw_df.shape, test_raw_df.shape

((1707695, 51), (401589, 50))

In [3]:
train_raw_df.head()

,channelGrouping,customDimensions,fullVisitorId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,...,hits_number_on_total_users_hits_62_days_before,pageviews_number_62_days_before,pageviews_number_on_total_users_pageviews_62_days_before,sum_user_revenue_31_days,mean_user_revenue_31_days,hits_number_31_days_before,hits_number_on_total_users_hits_31_days_before,pageviews_number_31_days_before,pageviews_number_on_total_users_pageviews_31_days_before,label
0,3,2,9499785259412240342,1,1472812272,6,0,False,5,106,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,3,4,3696906537737368442,1,1472856874,3,0,False,5,17,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,3,0,5688512451361930466,1,1472811524,3,0,False,2,10,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
3,3,0,9716453263501147451,1,1472823508,10,1,True,7,66,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,3,4,5330453591145100432,1,1472839619,3,0,False,5,90,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [4]:
total_df = pd.concat([train_raw_df, test_raw_df])
total_df.shape

(2109284, 51)

### Convert visitStartTime to different data features

In [5]:
def feature_engineering(df):
    
    df['full_date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['date'] = df['full_date'].dt.date
    df['day_of_week'] = df['full_date'].dt.dayofweek
    df['hour_of_day'] = df['full_date'].dt.hour
    df['day_of_month'] = df['full_date'].dt.day
    df['week_of_year'] = df['full_date'].dt.weekofyear
    
    df['month'] = df['full_date'].dt.month
    df['day'] = df['full_date'].dt.day
    
    df['month_unique_user_count'] = df.groupby('month')['fullVisitorId'].transform('nunique')
    df['day_unique_user_count'] = df.groupby('day')['fullVisitorId'].transform('nunique')
    
    #number of unique visitors per day of week (tells average amount of users per day of week)
    df['day_of_week_unique_user_count'] = df.groupby('day_of_week')['fullVisitorId'].transform('nunique')
    #number of unique visitors per week of year
    df['week_of_year_unique_user_count'] = df.groupby('week_of_year')['fullVisitorId'].transform('nunique')
    
    #todo: check what are these
    df['mean_hits_per_day'] = df.groupby('day')['totals_hits'].transform('mean')
    df['sum_hits_per_day'] = df.groupby('day')['totals_hits'].transform('sum')
    
    df['sum_pageviews_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('sum')
    df['count_pageviews_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('count')
    df['mean_pageviews_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_pageviews'].transform('mean')

    df['sum_pageviews_per_region'] = df.groupby('geoNetwork_region')['totals_pageviews'].transform('sum')
    df['count_pageviews_per_region'] = df.groupby('geoNetwork_region')['totals_pageviews'].transform('count')
    df['mean_pageviews_per_region'] = df.groupby('geoNetwork_region')['totals_pageviews'].transform('mean')
    
    df['sum_hits_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('sum')
    df['count_hits_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('count')
    df['mean_hits_per_network_domain'] = df.groupby('geoNetwork_networkDomain')['totals_hits'].transform('mean')

    df['sum_hits_per_region'] = df.groupby('geoNetwork_region')['totals_hits'].transform('sum')
    df['count_hits_per_region'] = df.groupby('geoNetwork_region')['totals_hits'].transform('count')
    df['mean_hits_per_region'] = df.groupby('geoNetwork_region')['totals_hits'].transform('mean')

    df['sum_hits_per_country'] = df.groupby('geoNetwork_country')['totals_hits'].transform('sum')
    df['count_hits_per_country'] = df.groupby('geoNetwork_country')['totals_hits'].transform('count')
    df['mean_hits_per_country'] = df.groupby('geoNetwork_country')['totals_hits'].transform('mean')
    
    #TODO: same analysis as before considering only users who brought revenues
    
    #these seem the most interesting columns, since they are referred to each single user (previous columns serve as a base to compute these ones)
    
    df['user_pageviews_sum'] = df.groupby('fullVisitorId')['totals_pageviews'].transform('sum')
    df['user_hits_sum'] = df.groupby('fullVisitorId')['totals_hits'].transform('sum')
    
    df['user_pageviews_count'] = df.groupby('fullVisitorId')['totals_pageviews'].transform('count')
    df['user_hits_count'] = df.groupby('fullVisitorId')['totals_hits'].transform('count')

    
    df['user_pageviews_sum_to_mean'] = df['user_pageviews_sum'] / df['user_pageviews_sum'].mean()
    df['user_hits_sum_to_mean'] = df['user_hits_sum'] / df['user_hits_sum'].mean()

    df['user_pageviews_to_region'] = df['user_pageviews_sum'] / df['mean_pageviews_per_region']
    df['user_hits_to_region'] = df['user_hits_sum'] / df['mean_hits_per_region']
    
    #OTHER FEATURES FOR EACH USER
    df['user_totals_pageviews_mean']=df.groupby(['fullVisitorId'])['totals_pageviews'].transform('mean')
    df['user_totals_pageviews_max']=df.groupby(['fullVisitorId'])['totals_pageviews'].transform('max')
    df['user_totals_pageviews_min']=df.groupby(['fullVisitorId'])['totals_pageviews'].transform('min')
    
    df['user_totals_hits_mean']=df.groupby(['fullVisitorId'])['totals_hits'].transform('mean')
    df['user_totals_hits_max']=df.groupby(['fullVisitorId'])['totals_hits'].transform('max')
    df['user_totals_hits_min']=df.groupby(['fullVisitorId'])['totals_hits'].transform('min')
    
    
    
    
    # Drop date column
    df.drop(['full_date'], axis=1, inplace=True)
    df.drop(['visitStartTime'], axis=1, inplace=True)
    return df

### Apply feature engineering

In [6]:
train_df = feature_engineering(train_raw_df)
test_df = feature_engineering(test_raw_df)
train_df.shape, test_df.shape

((1707695, 91), (401589, 90))

### Data preparation for catboost

In [7]:
cont_features = [
#                 'totals_hits',
#                 'totals_timeOnSite',
#                 'totals_pageviews', 
#                 'visitNumber',
#                 'totals_totalTransactionRevenue',
#                 'totals_transactionRevenue',
#                 'totals_transactions',
                'totals_sessionQualityDim',

                 
                 
                 # If a crafted feature is commented, it means that it is completely useless
                
#                 'month_unique_user_count',
#                 'day_unique_user_count',
#                 'day_of_week_unique_user_count',
#                 'week_of_year_unique_user_count',                
#                 'mean_hits_per_day',
#                 'sum_hits_per_day',
#                 'sum_pageviews_per_network_domain',
#                 'count_pageviews_per_network_domain',
#                 'mean_pageviews_per_network_domain',
#                 'sum_pageviews_per_region',
#                 'count_pageviews_per_region',
                'mean_pageviews_per_region',
#                 'sum_hits_per_network_domain',
#                 'count_hits_per_network_domain',
#                 'mean_hits_per_network_domain',
#                 'sum_hits_per_region',
#                 'count_hits_per_region',
#                 'mean_hits_per_region',
#                 'sum_hits_per_country',
#                 'count_hits_per_country',
#                 'mean_hits_per_country',
                'user_pageviews_sum',
                'user_hits_sum',
                'user_pageviews_count',
#                 'user_hits_count',
#                 'user_pageviews_sum_to_mean',
#                 'user_hits_sum_to_mean',
                'user_pageviews_to_region',
                'user_hits_to_region',
#                 'user_totals_pageviews_mean',
                'user_totals_pageviews_max',
#                 'user_totals_pageviews_min',
#                 'user_totals_hits_mean',
                'user_totals_hits_max',
#                 'user_totals_hits_min',
#                 'sum_user_revenue_62_days',
#                 'mean_user_revenue_62_days',
#                 'sum_user_revenue_31_days',
#                 'mean_user_revenue_31_days',
    
    
    
                'hits_number_62_days_before',
                'hits_number_on_total_users_hits_62_days_before',
                'pageviews_number_62_days_before',
                'pageviews_number_on_total_users_pageviews_62_days_before',
                'hits_number_31_days_before',
                'hits_number_on_total_users_hits_31_days_before',
                'pageviews_number_31_days_before',
#                 'pageviews_number_on_total_users_pageviews_31_days_before'
                ]

cat_features = [
                'fullVisitorId',
#                 'customDimensions',
#                 'channelGrouping', 
#                 'device_browser', 
#                 'device_deviceCategory',
#                 'device_operatingSystem', 
#                 'geoNetwork_continent',
#                 'geoNetwork_subContinent', 
#                 'trafficSource_medium', 
#                 'geoNetwork_country', 
#                 'geoNetwork_metro', 
#                 'geoNetwork_city', 
#                 'geoNetwork_region',
#                 'geoNetwork_networkDomain',
#                 'trafficSource_source', 
#                 'day_of_week', 
#                 'hour_of_day', 
#                 'day_of_month',
#                 'week_of_year', 
#                 'trafficSource_adContent', 
#                 'trafficSource_adwordsClickInfo.adNetworkType', 
#                 'trafficSource_adwordsClickInfo.gclId',
#                 'trafficSource_adwordsClickInfo.page', 
#                 'trafficSource_adwordsClickInfo.slot', 
#                 'trafficSource_keyword', 
#                 'trafficSource_referralPath',
#                 'trafficSource_campaign',
                #ex booleans
#                 'device_isMobile',
#                 'trafficSource_isTrueDirect',
#                 'trafficSource_adwordsClickInfo.isVideoAd',
#                 'totals_bounces',
#                 'totals_newVisits'
               ]

In [8]:
train = train_df[cont_features + cat_features]

# target
label = train_df['label']
train_y = label.values
log_y = np.log1p(train_y)

train.shape, log_y.shape

((1707695, 17), (1707695,))

In [9]:
#check if we took all the features except totals_totalTransactionRevenue (yep!)
for c in train_df.columns:
    if c not in train.columns:
        print(c)

channelGrouping
customDimensions
visitNumber
device_browser
device_deviceCategory
device_isMobile
device_operatingSystem
geoNetwork_city
geoNetwork_continent
geoNetwork_country
geoNetwork_metro
geoNetwork_networkDomain
geoNetwork_region
geoNetwork_subContinent
totals_bounces
totals_hits
totals_newVisits
totals_pageviews
totals_timeOnSite
totals_totalTransactionRevenue
totals_transactionRevenue
totals_transactions
trafficSource_adContent
trafficSource_adwordsClickInfo.adNetworkType
trafficSource_adwordsClickInfo.gclId
trafficSource_adwordsClickInfo.isVideoAd
trafficSource_adwordsClickInfo.page
trafficSource_adwordsClickInfo.slot
trafficSource_campaign
trafficSource_isTrueDirect
trafficSource_keyword
trafficSource_medium
trafficSource_referralPath
trafficSource_source
date
sum_user_revenue_62_days
mean_user_revenue_62_days
sum_user_revenue_31_days
mean_user_revenue_31_days
pageviews_number_on_total_users_pageviews_31_days_before
label
day_of_week
hour_of_day
day_of_month
week_of_year
mon

### Feature ranking with catboost

In [ ]:
#generate categorical column indices dinamically
cat_positions = []
for i in cat_features:
    cat_positions.append(train.columns.get_loc(i))
print(cat_positions)

In [ ]:
%%time
from catboost import CatBoostRegressor
from catboost import Pool

# necessary for feature importance ranking
pool = Pool(train, log_y, cat_features = cat_positions)


#fit catboost regressor
cat = CatBoostRegressor(
    learning_rate = 0.001,
    iterations = 150,
    max_depth = 2,
    
    random_seed = seed,
    loss_function = 'RMSE',
    eval_metric = 'RMSE',
    logging_level='Silent',
    task_type = 'CPU'
)


cat.fit(pool)

### Inspect features inside the model

In [ ]:
# get features importance ranking
# the individual importance values for each of the input features
features = train.columns
importances = np.asarray(cat.get_feature_importance(pool, fstr_type='FeatureImportance'))
indices = np.argsort(importances)# top features
plt.figure(figsize=(24, 12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# This peace of code works, but requires higher performange if the dataframe is the complete one
"""import shap

# load JS visualization code to notebook
shap.initjs()
shap_values = np.asarray(cat.get_feature_importance(pool, fstr_type='ShapValues'))

# summarize the effects of all the features
shap.summary_plot(
    shap_values[:,:pool.shape[1]], 
    features = train, 
    feature_names = train.columns,
    max_display = 100
)"""

### Prepare the dataframe for the final export

In [10]:
final_train_df = train_df[cont_features + cat_features]
final_train_df = pd.concat([final_train_df, train_df.date], axis=1)

final_train_df = pd.concat([final_train_df, label], axis=1)
final_test_df = test_df[cont_features + cat_features]
final_train_df.shape, final_test_df.shape

((1707695, 19), (401589, 17))

In [11]:
final_train_df.head()

,totals_sessionQualityDim,mean_pageviews_per_region,user_pageviews_sum,user_hits_sum,user_pageviews_count,user_pageviews_to_region,user_hits_to_region,user_totals_pageviews_max,user_totals_hits_max,hits_number_62_days_before,hits_number_on_total_users_hits_62_days_before,pageviews_number_62_days_before,pageviews_number_on_total_users_pageviews_62_days_before,hits_number_31_days_before,hits_number_on_total_users_hits_31_days_before,pageviews_number_31_days_before,fullVisitorId,date,label
0,0.0,3.402187,1.0,1,1,0.293929,0.246817,1.0,1,0,0.0,0.0,0.0,0,0.0,0.0,9499785259412240342,2016-09-02,0.0
1,0.0,2.723217,1.0,1,1,0.367213,0.318906,1.0,1,0,0.0,0.0,0.0,0,0.0,0.0,3696906537737368442,2016-09-02,0.0
2,0.0,2.628769,1.0,1,1,0.380406,0.322538,1.0,1,0,0.0,0.0,0.0,0,0.0,0.0,5688512451361930466,2016-09-02,0.0
3,0.0,2.537001,1.0,1,1,0.394166,0.325649,1.0,1,0,0.0,0.0,0.0,0,0.0,0.0,9716453263501147451,2016-09-02,0.0
4,0.0,2.814610,1.0,1,1,0.355289,0.301940,1.0,1,0,0.0,0.0,0.0,0,0.0,0.0,5330453591145100432,2016-09-02,0.0


In [12]:
final_test_df.head()

,totals_sessionQualityDim,mean_pageviews_per_region,user_pageviews_sum,user_hits_sum,user_pageviews_count,user_pageviews_to_region,user_hits_to_region,user_totals_pageviews_max,user_totals_hits_max,hits_number_62_days_before,hits_number_on_total_users_hits_62_days_before,pageviews_number_62_days_before,pageviews_number_on_total_users_pageviews_62_days_before,hits_number_31_days_before,hits_number_on_total_users_hits_31_days_before,pageviews_number_31_days_before,fullVisitorId
0,1,2.995610,5.0,6,2,1.669109,1.625595,3.0,4,0,0.000000,0.0,0.000000,0,0.000000,0.0,7460955084541987166
1,1,5.594905,68.0,76,15,12.153915,10.933027,11.0,13,151,0.000188,130.0,0.000196,70,0.000183,55.0,460252456180441002
2,1,3.934772,5.0,7,2,1.270722,1.432860,3.0,4,0,0.000000,0.0,0.000000,0,0.000000,0.0,3461808543879602873
3,1,5.416863,13.0,16,3,2.399913,2.323793,6.0,7,0,0.000000,0.0,0.000000,0,0.000000,0.0,975129477712150630
4,1,5.594905,4.0,5,1,0.714936,0.719278,4.0,5,0,0.000000,0.0,0.000000,0,0.000000,0.0,8381672768065729990


### Export Dataframes

In [13]:
final_train_df.to_csv('(5)final_train.csv', index = False)
final_test_df.to_csv('(5)final_test.csv', index = False)